In [1]:
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from azureml.automl.core.featurization import FeaturizationConfig

In [19]:
print("This notebook was created using version 1.32.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.32.0 of the Azure ML SDK
You are currently using version 1.32.0 of the Azure ML SDK



As part of the setup you have already created a Workspace.
To run AutoML, you also need to create an Experiment. 
An Experiment corresponds to a prediction problem you are trying to solve,
while a Run corresponds to a specific approach to the problem.

In [2]:
ws = Workspace.get("myworkspace", subscription_id='ff711122-6294-4fad-9d1f-bf505a51fc42',
               resource_group='mlproject',
               location='westus2')

In [3]:
ws.datasets

{'titanic': DatasetRegistration(id='2cdbeea0-cef8-43d8-bf62-b32e27ec0e55', name='titanic', version=1, description='', tags={})}

In [5]:
iris_df=Dataset.get_by_name(workspace=ws,name="titanic")

# Convert to Pandas DataFrame

In [7]:
titanic= Dataset.get_by_name(workspace=ws, name="titanic")

In [9]:
pd_df=titanic.to_pandas_dataframe()

In [10]:
pd_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,None,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,None,S


In [11]:
pd_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [13]:
X_train, X_test = train_test_split(pd_df, test_size=0.2, random_state=42)

# Compute
You will need to create a compute target for your AutoML run. In this tutorial, you create AmlCompute as your training compute resource.

Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

Creation of AmlCompute takes approximately 5 minutes.
If the AmlCompute with that name is already in your workspace this code will skip the creation process. As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read this article on the default limits and how to request more quota.

In [20]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "oj-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2',
                                                           max_nodes=1)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Upload some data to the Datastore
The Machine Learning service workspace, is paired with the storage account, which contains the default data store. We will use it to upload the train and test data and create tabular datasets for training and testing. A tabular dataset defines a series of lazily-evaluated, immutable operations to load data from the data source into tabular representation.

In [27]:
# train.to_csv (r'./dominicks_OJ_train.csv', index = None, header=True)
# test.to_csv (r'./dominicks_OJ_test.csv', index = None, header=True)
path='./titanic2.csv'
datastore=ws.get_default_datastore()
datastore.upload_files(files=path, target_path='dataset/',overwrite=True,show_progress=True)

UserErrorException: UserErrorException:
	Message: '.' does not point to a file. Please upload the file to cloud first if running in a cloud notebook.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "'.' does not point to a file. Please upload the file to cloud first if running in a cloud notebook."
    }
}

# Auto ML

In [14]:
automl_settings = {
    "iteration_timeout_minutes": 2,
    "experiment_timeout_minutes": 15,
    "enable_early_stopping": True,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "n_cross_validations": 5,
}

# Specift Auto ML Config

In [28]:
automl_config = AutoMLConfig(task='classification',
    debug_log='automl_errors.log',
    training_data=X_train,
    label_column_name="Survived",
    **automl_settings)

# Create an Experiment for the Model

In [29]:
experiment = Experiment(ws, "titanic_sdk_experiment")

In [30]:
run = experiment.submit(automl_config, show_output=True)

ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: C:\Users\User\anaconda3\lib\site-packages\azureml\automl\core\validated_win32_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Required version/Installed version
azure-core<=1.15.0/azure-core 1.16.0
azure-mgmt-core<=1.2.2/azure-mgmt-core 1.3.0
jeepney<=0.6.0/jeepney 0.7.0
numpy<=1.18.5/numpy 1.19.0
pandas<=0.25.3/pandas 1.0.0
ruamel.yaml.clib<=0.2.4/ruamel.yaml.clib 0.2.6
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: C:\\Users\\User\\anaconda3\\lib\\site-packages\\azureml\\automl\\core\\validated_win32_requirements.txt. Alternatively, use remote target to avoid dependency management. \nRequired version/Installed version\nazure-core<=1.15.0/azure-cor